In [ ]:
# !pip install langchain
# !pip install langchain-community
# !pip install openai
# !pip install unstructured
# !pip install "unstructured[docx]"

In [ ]:
from bs4 import BeautifulSoup  # HTML 태그를 제거하기 위한 라이브러리
import re  # 정규 표현식을 사용하여 특수 문자를 제거하기 위한 모듈
import requests  # HTTP 요청을 보내기 위한 라이브러리
from langchain.document_loaders import TextLoader  # 텍스트 파일을 로드하기 위한 LangChain의 로더
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 텍스트를 분할하는 모듈

class McDonaldsMenu:
    def __init__(self):
        # 메뉴 데이터를 가져오기 위한 API의 URL과 헤더 설정
        self.url = "https://www.mcdonalds.co.kr/kor/menu/listContent.do"
        # HTTP 요청을 보낼 때 사용할 헤더
        self.headers = {
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "User-Agent": "Mozilla/5.0"  # 브라우저를 흉내 내기 위해 User-Agent 설정
        }

    def fetch_menu_data(self, sub_category_seq, page_num):
        """
        카테고리와 페이지별로 메뉴 데이터를 가져오는 함수.
        sub_category_seq: 카테고리 번호
        page_num: 페이지 번호
        """
        # POST 요청에 필요한 데이터 (카테고리 번호와 페이지 번호)
        data = {
            "page": page_num,
            "sub_category_seq": sub_category_seq
        }

        try:
            # POST 요청을 보내고, 응답을 받음
            response = requests.post(self.url, headers=self.headers, data=data)
            # 응답에서 HTTP 오류가 있으면 예외를 발생시킴
            response.raise_for_status()  # 4xx, 5xx 오류 처리

            # JSON 응답에서 'list' 키에 해당하는 메뉴 리스트 반환
            return response.json().get('list', [])  # 'list'가 없으면 빈 리스트 반환
        except requests.exceptions.RequestException as e:
            # 요청에 실패한 경우 예외를 출력하고 빈 리스트 반환
            print(f"API 요청 실패: {e}")
            return []

    def clean_text(self, text):
        """
        HTML 태그와 특수 문자를 제거하는 함수.
        text: 처리할 텍스트
        """
        # BeautifulSoup을 사용하여 HTML 태그 제거
        cleaned_text = BeautifulSoup(text, "html.parser").get_text()

        # 정규 표현식을 사용하여 ™, ® 등의 특수 문자를 제거
        cleaned_text = re.sub(r'[™®]', '', cleaned_text)
        return cleaned_text

    def save_menu_to_file(self, all_menu_items):
        """
        모든 메뉴 데이터를 파일에 저장하는 함수.
        all_menu_items: API를 통해 가져온 메뉴 데이터 리스트
        """
        # 메뉴 데이터를 "/content/drive/MyDrive/Aiffel/Project/menu_data.txt"에 저장
        with open("/content/drive/MyDrive/Aiffel/Project/menu_data.txt", "w", encoding="utf-8") as f:
            for item in all_menu_items:
                # 메뉴 이름을 전처리 (HTML 태그 및 특수 문자 제거)
                kor_name = self.clean_text(item.get('kor_name', ''))
                eng_name = self.clean_text(item.get('eng_name', ''))
                # 한글 메뉴명 / 영어 메뉴명 형식으로 파일에 쓰기
                f.write(f"{kor_name} / {eng_name}\n")
        print("메뉴 데이터를 파일에 저장했습니다.")


# McDonaldsMenu 클래스의 인스턴스 생성
menu = McDonaldsMenu()
all_menu_items = []  # 모든 메뉴 데이터를 저장할 리스트

# 카테고리와 페이지 번호에 따라 데이터를 가져옴
# 예시에서는 1번과 2번 카테고리에서 각각 1페이지부터 5페이지까지 데이터를 가져옴
for sub_category_seq in [1, 2]:  # 필요한 카테고리 번호 입력
    for page_num in range(1, 6):  # 필요한 페이지 번호 입력
        # 각 카테고리 및 페이지별로 API 호출
        menu_data = menu.fetch_menu_data(sub_category_seq, page_num)
        if not menu_data:  # 더 이상 데이터가 없으면 루프 종료
            break
        all_menu_items.extend(menu_data)  # 가져온 데이터를 리스트에 추가

# 가져온 메뉴 데이터를 파일에 저장 (HTML 태그 및 특수 문자를 제거한 후 저장)
menu.save_menu_to_file(all_menu_items)